In [1]:
import os
import re
import pandas as pd
import numpy as np
labels_arq = './base_treino.csv'
data_dir = './base_treino/'

In [2]:
labels = pd.read_csv(labels_arq, sep=';', header=None)
labels.columns = ['id', 'url', '_label_']
labels.set_index('id', inplace = True)

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels['_label_'] = le.fit_transform(labels._label_)

In [4]:
labels = labels.sample(frac = 0.2)

In [5]:
os.chdir(data_dir)
lista_arquivos = os.listdir('.')

In [6]:
df_features = []
for idx, home in enumerate(lista_arquivos):
    with open(home, 'rb') as fp:
        df_features.append((home, fp.read()))

In [7]:
df_features = pd.DataFrame(df_features, )
df_features.columns = ['url', 'conteudo']

df_features = pd.merge(df_features, labels)
df_features.drop('url', axis = 1, inplace = True)

In [8]:
init_size = len(df_features['_label_'])
df_features = df_features[df_features.apply(lambda x: (not 'Internal Server Error' in x.conteudo.decode('utf-8')) & (not '404' in x.conteudo.decode('utf-8')), axis = 1)]

In [9]:
RE_D = re.compile('\d')
def nao_tem_numero(string):
    return not bool(RE_D.search(string))

In [10]:
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer('portuguese')

In [11]:
from bs4 import BeautifulSoup
def limpeza_numeros(txt):
    exclusao = set('1234567890')
    lista_retorno = [value for value in  txt.split(' ') if (len(set(value) & exclusao) is 0)]
    return ' '.join(lista_retorno)

In [12]:
def limpeza_beatiful_soup(html):
    soup = BeautifulSoup(html, 'html.parser')
    for script in soup(["script", "style"]):
        script.extract()    
    text = soup.get_text()
    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)
    text = limpeza_numeros(text.replace('\n',' ').replace('\t', ' ').replace('.','').replace('-','').replace('/',''))
    return text.lower()

In [13]:
df_features['conteudo'] = df_features.conteudo.apply(lambda x: limpeza_beatiful_soup(x))

In [14]:
stop_words_list = ['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'Em', 'um', 'para', 'é', 'com', 
                   'não', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'foi', 
                   'ao', 'ele', 'das', 'tem', 'à', 'seu', 'sua', 'ou', 'ser', 'quando', 'muito', 'há', 
                   'nos', 'já', 'está', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 
                   'era', 'depois', 'sem', 'mesmo', 'aos', 'ter', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 
                   'estão', 'você', 'tinha', 'foram', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 
                   'têm', 'numa', 'pelos', 'elas', 'havia', 'seja', 'qual', 'será', 'nós', 'tenho', 'lhe', 
                   'deles', 'essas', 'esses', 'pelas', 'este', 'fosse', 'dele', 'tu', 'te', 'vocês', 'vos', 
                   'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 
                   'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 
                   'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 
                   'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 
                   'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessemos', 'estivessem', 'estiver', 
                   'estivermos', 'estiverem', 'hei', 'há', 'havemos', 'hão', 'houve', 'houvemos', 'houveram', 
                   'houvera', 'houvéramos', 'haja', 'hajamos', 'hajam', 'houvesse', 'houvéssemos', 'houvessem', 
                   'houver', 'houvermos', 'houverem', 'houverei', 'houverá', 'houveremos', 'houverão', 
                   'houveria', 'houveríamos', 'houveriam', 'sou', 'somos', 'são', 'era', 'éramos', 'eram', 
                   'fui', 'foi', 'fomos', 'foram', 'fora', 'fôramos', 'seja', 'sejamos', 'sejam', 'fosse', 
                   'fôssemos', 'fossem', 'for', 'formos', 'forem', 'serei', 'será', 'seremos', 'serão', 'seria', 
                   'seríamos', 'seriam', 'tenho', 'tem', 'temos', 'tém', 'tinha', 'tínhamos', 'tinham', 'tive', 
                   'teve', 'tivemos', 'tiveram', 'tivera', 'tivéramos', 'tenha', 'tenhamos', 'tenham', 'tivesse', 
                   'tivéssemos', 'tivessem', 'tiver', 'tivermos', 'tiverem', 'terei', 'terá', 'teremos', 'terão', 
                   'teria', 'teríamos', 'teriam', 'a', 'about', 'above', 'after', 'again', 'against', 'all', 
                   'am', 'an', 'and', 'any', 'are', 'arent', 'as', 'at', 'be', 'because', 'been', 'before', 
                   'being', 'below', 'between', 'both', 'but', 'by', 'cant', 'cannot', 'could', 'couldnt', 
                   'did', 'didnt', 'do', 'does', 'doesnt', 'doing', 'dont', 'down', 'during', 'each', 'few', 
                   'for', 'from', 'further', 'had', 'hadnt', 'has', 'hasnt', 'have', 'havent', 'having', 'he', 
                   'hed', 'hes', 'her', 'here', 'heres', 'hers', 'herself', 'him', 'himself', 'his', 'how', 
                   'hows', 'i', 'id', 'im', 'ive', 'if', 'in', 'into', 'is', 'isnt', 'it', 'its', 'its', 
                   'itself', 'lets', 'me', 'more', 'most', 'mustnt', 'my', 'myself', 'no', 'nor', 'not', 'of', 
                   'off', 'on', 'once', 'only', 'or', 'other', 'ought', 'our', 'ours', 'ourselves', 'out', 'over', 
                   'own', 'same', 'shant', 'she', 'shed', 'shell', 'shes', 'should', 'shouldnt', 'so', 'some', 
                   'such', 'than', 'that', 'thats', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 
                   'theres', 'these', 'they', 'theyd', 'theyll', 'theyre', 'theyve', 'this', 'those', 'through', 
                   'to', 'too', 'under', 'until', 'up', 'very', 'was', 'wasnt', 'we', 'wed', 'well', 'were', 
                   'weve', 'were', 'werent', 'what', 'whats', 'when', 'whens', 'where', 'wheres', 'which', 'while', 
                   'who', 'whos', 'whom', 'why', 'whys', 'with', 'wont', 'would', 'wouldnt', 'you', 'youd', 'youll', 
                   'youre', 'youve', 'your', 'yours', 'yourself', 'yourselves']

In [19]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
cv = CountVectorizer(strip_accents='ascii', min_df = 40, max_df = 0.5, stop_words = stop_words_list)
vec_cv = cv
df_tokens_cv = pd.DataFrame(vec_cv.fit_transform(df_features.conteudo).toarray(),
                         index = df_features.index).rename(columns = {v:k for k,v in vec_cv.vocabulary_.items()})
df_features_cv = pd.concat([df_features, df_tokens_cv], axis = 1).drop('conteudo', axis = 1)

In [20]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [21]:
batch_size = 512
epochs = 5
X_cv = df_features_cv.drop('_label_', axis = 1).values
y = df_features_cv._label_.values
train_size = 0.8
x_train_cv, x_test_cv = X_cv[:int(len(X_cv)*train_size)+1],X_cv[int(len(X_cv)*train_size)+1: len(X_cv)]
y_train, y_test = y[:int(len(y)*train_size)+1],y[int(len(y)*train_size)+1: len(y)]
num_classes = 10
y_nn_train = keras.utils.to_categorical(y_train, num_classes)
y_nn_test = keras.utils.to_categorical(y_test, num_classes)

In [22]:
def _get_model_():
    num_features = len(x_train_cv[0])
    model = Sequential()
    model.add(Dense(int(num_features/4),activation='relu',input_shape=(num_features,),name = 'layer_1'))   
    model.add(Dropout(0.01))
    model.add(Dense(int(num_features/8), activation='sigmoid', name = 'layer_2'))
    model.add(Dropout(0.01))
    model.add(Dense(int(num_features/64), activation='sigmoid', name = 'layer_3'))
    model.add(Dropout(0.01))
    model.add(Dense(10, activation='softmax', name = 'output_layer'), )
    return model
model = _get_model_()

In [23]:
model.compile(loss = 'categorical_crossentropy',optimizer = 'adam',metrics = ['accuracy'])

In [24]:
history = model.fit(x_train_cv, y_nn_train,
                    batch_size = batch_size,
                    epochs = epochs,   # baixo aprende pouco, alto pode causar overfitting
                    verbose = 1,
                    validation_data = (x_test_cv, y_nn_test))

Train on 801 samples, validate on 199 samples
Epoch 1/5
801/801 [==============================] - 0s - loss: 2.3302 - acc: 0.4195 - val_loss: 2.3327 - val_acc: 0.4020
Epoch 2/5
801/801 [==============================] - 0s - loss: 2.3038 - acc: 0.4220 - val_loss: 2.3121 - val_acc: 0.4070
Epoch 3/5
801/801 [==============================] - 0s - loss: 2.2816 - acc: 0.4345 - val_loss: 2.2935 - val_acc: 0.4070
Epoch 4/5
801/801 [==============================] - 0s - loss: 2.2576 - acc: 0.4482 - val_loss: 2.2762 - val_acc: 0.4070
Epoch 5/5
801/801 [==============================] - 0s - loss: 2.2392 - acc: 0.4507 - val_loss: 2.2604 - val_acc: 0.4121


In [25]:
score = model.evaluate(x_test_cv, y_nn_test, verbose=0)

In [26]:
X_cv = df_features_cv.drop('_label_', axis = 1).values
x_base = X_cv[:int(len(X_cv))+1]

In [27]:
y_pred = model.predict(x_base)
y_pred = pd.DataFrame(data=y_pred)

In [28]:
y_pred['Result_Keras'] = y_pred[[0,1,2,3,4,5,6,7,8,9]].idxmax(axis=1)

In [29]:
Result = pd.concat([df_features_cv.reset_index(), y_pred], axis = 1)

In [33]:
model = _get_model_()
model.save('/home/felipe/webex/modelo_sinistro.h5')